# Benchmark Chart
This notebook reads .codspeed benchmarks generated in the root of conda-libmamba-solver with `pytest -k benchmark --codspeed` e.g. and renders `test_traversal_algorithm_benchmarks` as a plotly chart.

`jupyter nbconvert --clear-output --inplace <file>.ipynb` clears output from the file for better revision control.

In [ ]:
import pandas as pd
import plotly.express as px
import json
from pathlib import Path
import os

In [ ]:
runs = Path(".codspeed").glob("*.json")
most_recent = sorted(runs, key=lambda f: os.stat(f).st_mtime)[-1]
print(most_recent)

In [ ]:
SCENARIOS = [
    "python",
    "data_science_basic",
    "data_science_ml",
    "web_development",
    "scientific_computing",
    "devops_automation",
    "vaex",
]

benchmarks = json.loads(most_recent.read_text())["benchmarks"]
traversal_benchmarks = []
for benchmark in benchmarks:
    name = benchmark["name"]
    for scenario in SCENARIOS:
        name = name.replace(scenario.replace("_", "-"), scenario.replace("-", "_"))
    if not "test_traversal_algorithm_benchmarks" in name:
        continue
    stats = benchmark["stats"]
    assert len(name.split("[")[-1].strip("]").split("-")) == 4
    facets = dict(
        zip(["main", "scenario", "alg", "cache"], name.split("[")[-1].strip("]").split("-"))
    )
    stats["min_s"] = stats["min_ns"] / 1e9
    traversal_benchmarks.append(stats | facets)

In [ ]:
df = pd.DataFrame(traversal_benchmarks)

In [ ]:
fig = px.bar(
    df,
    x="main",
    y="min_s",
    color="alg",
    barmode="group",
    facet_row="scenario",
    facet_col="cache",
    category_orders={
        "cache": ["cold", "warm", "lukewarm"],
        "alg": ["bfs", "pipelined", "httpx"],
        "main": ["main"],
    },
    width=1000,
    height=1600,
)
fig.show()

In [ ]:
df[df["alg"] == "pipelined"].sum()["min_ns"] / df[df["alg"] == "httpx"].sum()["min_ns"]

In [ ]:
df[df["alg"] == "bfs"].sum()["min_ns"] / df[df["alg"] == "httpx"].sum()["min_ns"]